In [ ]:
# execute script
# multiple processes start outside of notebook
from os.path import abspath, join
from subprocess import Popen, PIPE

script_path = abspath('')
script_exec_path = join(script_path,'compare-folders.py')
print("To execute:\n" + script_exec_path)

process = Popen(
    ['py', script_exec_path,
     '-d',
     'E:\\' 
    ],
    stdout=PIPE, stderr=PIPE, 
    text=True  # Ensures strings instead of bytes
)

stdout, stderr = process.communicate()

print("Output:\n", stdout)
print("Error:\n", stderr)
# output: csv file

In [ ]:
from os.path import isdir, abspath
from os import remove, listdir

script_path = abspath('')
keyword = "processed"
in_path = ""
for entry in listdir(script_path):
    if not isdir(entry):
        if ".csv" in entry:
            if keyword in entry:
                remove(entry)
            else:
                in_path = entry
        
print(in_path)
# output: in_path stores location of csv

In [ ]:
# post process csv file
# output: two csv-s
from ast import literal_eval
from csv import DictReader, field_size_limit
from os.path import splitdrive
from sys import maxsize

field_size_limit(maxsize)

sep = '.'
p,ext = in_path.split(sep)
duplicates_path = p + "-" + keyword + sep + ext
singles_path = p + "-" + keyword + "-" + "single" + sep + ext
print(singles_path, duplicates_path)

with open(duplicates_path, mode = "w", encoding="utf-8") as d_file:
    with open(singles_path, mode = "w", encoding="utf-8") as s_file:
        with open(in_path, mode = "r", encoding="utf-8") as in_file:
            reader = DictReader(in_file)
            for i, row in enumerate(reader):
                value = literal_eval(row["paths"])
                if "desktop.ini" in value[0]: # ignore system files
                    continue
            
                # write duplicate paths once:
                s = set()
                for entry in value:
                    drive, entry = splitdrive(entry)
                    s.add(entry)

                if len(s) <= 1:
                    file = s_file
                else: # >=2 are duplicates
                    file = d_file

                file.write(f"{i}\n")
                for entry in s:
                    file.write(f"{join(drive, entry)}\n")


In [ ]:
# post process csv file
# read csv file and remove duplicates with user interogations

from ast import literal_eval
from ctypes import windll
from csv import DictReader, field_size_limit
from os import remove
from os.path import splitdrive, join
from sys import maxsize

# Define constants for the MessageBox
MB_YESNO = 4
MB_RET_IDYES = 6
MB_ICON = 0x40

def execute():
    
    with open(in_path, mode = "r", encoding="utf-8") as in_file:
        reader = DictReader(in_file)
        for row in reader:
            value = literal_eval(row["paths"])
            if "desktop.ini" in value[0]: # ignore system files
                continue
        
            # write duplicate paths once:
            s = set()
            for entry in value:
                drive, entry = splitdrive(entry)
                s.add(entry)

            total = len(s)
            if len(s) > 1: # if there's at least one duplicated file
                
                file_all_paths = ""
                s = sorted(s)
                for i, entry in enumerate(s, start=1):
                    file_all_paths += str(i) + "\t" + entry.lower() + "\n"

                for i, entry in enumerate(s, start=1):
                    file_path = join(drive, entry).lower()
                    r = windll.user32.MessageBoxW(0,
                                f"All paths: {total}\n{file_all_paths}\n" + 
                                f"Delete {i}:\n {file_path} ?",
                                "Confirmation",  MB_YESNO)
                    try:
                        if r == MB_RET_IDYES:
                            print(f"Deleted: {file_path}")
                            remove(file_path)
                    except:
                        pass
            
if __name__ == "__main__":

    field_size_limit(maxsize) 

    execute()
